# 0. Premilinary Works
How big is the file, number of rows, etc.

In [1]:
cd ../data

/home/angelrps/git/MasterDataScience_FinalProject/data


In [2]:
!ls -Rl
# 2017_Yellow_Taxi_Trip_Data.csv is 10532305815 bytes (9,80 GB)
# 2018_Yellow_Taxi_Trip_Data.csv is 10428263736 bytes (9,71 GB)
# 2019_Yellow_Taxi_Trip_Data.csv is 8197837930 bytes (7,63 GB)

.:
total 3660944
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rw-rw-rw- 1 angelrps angelrps   13400119 Jun 23 09:34 Data_Cleaned_To_Model.csv
-rw-rw-rw- 1 angelrps angelrps   24377425 Jun 20 06:26 Data_Taxis_2017_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26220204 Jun 20 04:20 Data_Taxis_2018_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26759445 Jun 20 00:08 Data_Taxis_2019_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps    3755058 Jun 22 10:01 NY_taxi_zones.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionar

In [4]:
# I am dealing with very heavy files so I will compress them (bz2) to work with them
# and remove the heavy .csv from my drive
#!bzip2 2017_Yellow_Taxi_Trip_Data.csv 2018_Yellow_Taxi_Trip_Data.csv 2019_Yellow_Taxi_Trip_Data.csv

In [5]:
!ls -Rl

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [5]:
# Count Number of Lines: IT TAKES TIME, JUST BE AWARE BEFORE YOU UNCOMMENT
# !bzcat -c ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
# !bzcat -c ./2018_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
# !bzcat -c ./2017_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l

# 2017_...113.496.875 lines
# 2018_...112.234.627 lines
# 2019_...84.399.020 lines
# The process took about 15 min to calculate the lines

84399020
112234627
113496875


In [11]:
!bzcat ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | head -n 5

# Conclusions:
# 1. There is a header on the first line
# 2. The separator is ','

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,04/29/2019 11:31:03 AM,04/29/2019 11:31:03 AM,1,0,1,N,231,264,2,2.5,2.5,0.5,0,0,0.3,5.8,2.5
2,04/29/2019 10:52:08 AM,04/29/2019 11:06:18 AM,1,1.01,1,N,186,230,1,10,0,0.5,2.66,0,0.3,15.96,2.5
2,04/29/2019 11:28:40 AM,04/29/2019 11:33:01 AM,1,0.82,1,N,238,151,1,5,0,0.5,1.66,0,0.3,9.96,2.5
1,04/29/2019 11:28:06 AM,04/29/2019 12:07:32 PM,1,12.1,1,N,138,88,1,38.5,2.5,0.5,10.4,0,0.3,52.2,2.5

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./2019_Yellow_Taxi_Trip_Data.csv.bz2, output file = (stdout)


### 1. Clean up dataset taking just a sample

In [1]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

The following cleaning actions are based on the dataset exploration made in the notebok 'Data_Analysis.ipynb'

1. Import part of the csv so the memory can handle it.
2. Let's remove trips under 0.06 (100m). I consider them either errors or not relevant
3. Remove negative 'fare_amount' or <1$. I consider them errors or not relevant.
4. Select the two columns of interest:
    - 'pickup_datetime'
    - 'PULocationID'
5. Drop unwanted years:<br>
Eventhough the dataset states the year, I have detected rows with wrong years so I´ll filter them out.
6. Groupby Hour and LocationID.
7. Break down datetime in other useful variables:
    - year
    - month
    - day
    - week
    - dayofweek
    - isweekend
    - isholiday

### Create Data Frame with Manhattan zones

In [22]:
# 1. Import Location and Borough columns form NY TAXI ZONES dataset
dfzones = pd.read_csv('../data/NY_taxi_zones.csv', sep=',',
                      usecols=['LocationID', 'borough'])

# 2. Filter Manhattan zones
dfManZon = dfzones[dfzones['borough']=='Manhattan']

# 3. Get unique Zone values
dfManZon = dfManZon.drop_duplicates('LocationID')

# 4. Drop Borough column to keep just LocationID values and sort them
dfManZon = dfManZon.drop(['borough'], axis=1).sort_values(by='LocationID').reset_index()


dfManZon_array = dfManZon.iloc[:,0].values
dfManZon_array
# There are 67 zones in Manhattan

array([  3,  12,  13,   9,  40,  43,  44,  41,  47,  49,  68,  75,  78,
        73,  83,  87,  89, 100, 103, 107, 113, 114, 116, 120,  90, 128,
       129, 141, 138, 142, 143, 146, 147, 151, 132, 144, 152, 158, 160,
       161, 162, 163, 166, 164, 185, 193, 205, 210, 212, 221, 228, 229,
       230, 232, 233, 234, 236, 237, 238, 231, 245, 246, 248, 251, 261,
       262, 244])

In [23]:
# RECAP

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
year = 2019
dfsample = pd.read_csv('../data/'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)
# dfsample[dfsample['trip_distance']<0.06].index # this should be none

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)
#dfsample[dfsample['fare_amount']<1] # this should be none

# 4. Filter just Manhattan pickups
dfsample = dfsample.loc[dfsample['PULocationID'].isin(dfManZon_array)]
#dfsample['PULocationID'].unique().shape

# 5. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']].copy()
#dfsample.head()

# 6. Drop unwanted years
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(dt(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(dt(year+1,1,1))].index, inplace=True)

# 7. Groupby HOUR and LocationID
    # set time period index
dfsample.set_index('pickup_datetime', inplace=True) # Set timestamp as index
dfsample = dfsample.to_period("H") # convert DATE to 1 hour PERIODS
    # Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
    # Create new column to count Number of Pickups per LocationID
dfsample['NoOfPickups'] =1
    # Groupby Datetime and Location to get Pickups sum()
dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
#dfsample.head(50)

# 8. Create new variables from datetime
dfsample['year'] = dfsample['pickup_datetime'].dt.year
dfsample['month'] = dfsample['pickup_datetime'].dt.month
dfsample['day'] = dfsample['pickup_datetime'].dt.day
dfsample['hour'] = dfsample['pickup_datetime'].dt.hour
dfsample['week'] = dfsample['pickup_datetime'].dt.week
dfsample['dayofweek'] = dfsample['pickup_datetime'].dt.dayofweek
    # 8.1 create variable 'isweekend'
mask = (dfsample['dayofweek'] == 5) | (dfsample['dayofweek'] == 6)
dfsample['isweekend'] = np.where(mask, 1, 0)
#dfsample.sample(50)

    # 8.2 Create date time index calendar
drange = pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31')
cal = calendar()
holidays = cal.holidays(start=drange.min(), end=drange.max())
    # 8.3 create new column 'IsHolidays'
dfsample['IsHoliday'] = dfsample['pickup_datetime'].isin(holidays).astype(int)

print (dfsample.tail())
print(dfsample.info())
print(holidays)

      pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  \
136  2019-04-29 13:00           229            1  2019      4   29    13   
137  2019-04-29 13:00           230            1  2019      4   29    13   
138  2019-04-29 13:00           231            1  2019      4   29    13   
139  2019-04-29 13:00           234            1  2019      4   29    13   
140  2019-04-29 13:00           236            2  2019      4   29    13   

     week  dayofweek  isweekend  IsHoliday  
136    18          0          0          0  
137    18          0          0          0  
138    18          0          0          0  
139    18          0          0          0  
140    18          0          0          0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 11 columns):
pickup_datetime    141 non-null period[H]
PULocationID       141 non-null int64
NoOfPickups        141 non-null int64
year               141 non-null int64
month       

In [9]:
dfsample.shape

(141, 11)

### 2. Test Big Data approach on a sample

In [24]:
# DEFINE FUNCTION TO CLEAN DATASET
def clean_data(dfinput, yearinput, manhattan_zones_array):
    DistIndex_to_drop = dfinput[dfinput['trip_distance'] < 0.06].index
    dfinput.drop(DistIndex_to_drop, inplace=True)

    FareIndex_to_drop = dfinput[dfinput['fare_amount']<1].index
    dfinput.drop(FareIndex_to_drop, inplace=True)
    
    dfinput = dfinput.loc[dfinput['PULocationID'].isin(manhattan_zones_array)]

    dfinput = dfinput[['pickup_datetime','PULocationID']].copy()

    dfinput.drop(dfinput[dfinput['pickup_datetime'] < pd.Timestamp(dt(yearinput,1,1))].index, inplace=True)
    dfinput.drop(dfinput[dfinput['pickup_datetime'] >= pd.Timestamp(dt(yearinput+1,1,1))].index, inplace=True)

    dfinput.set_index('pickup_datetime', inplace=True) # Set timestamp as index
    dfinput = dfinput.to_period("H") # convert DATE to 1 hour PERIODS
    dfinput.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
    dfinput['NoOfPickups'] =1
    
    dfinput=dfinput.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    
    return dfinput

# DEFINE FUNCTION TO ADD DATE FEATURES TO DATASET
def add_date_features(dfinput, yearinput):
    
    dfinput['year'] = dfinput['pickup_datetime'].dt.year
    dfinput['month'] = dfinput['pickup_datetime'].dt.month
    dfinput['day'] = dfinput['pickup_datetime'].dt.day
    dfinput['hour'] = dfinput['pickup_datetime'].dt.hour
    dfinput['week'] = dfinput['pickup_datetime'].dt.week
    dfinput['dayofweek'] = dfinput['pickup_datetime'].dt.dayofweek

    mask = (dfinput['dayofweek'] == 5) | (dfinput['dayofweek'] == 6)
    dfinput['isweekend'] = np.where(mask, 1, 0)

    drange = pd.date_range(start=str(yearinput)+'-01-01', end=str(yearinput)+'-12-31')
    cal = calendar()
    holidays = cal.holidays(start=drange.min(), end=drange.max())
        # 6.3 create new column 'IsHolidays'
    dfinput['IsHoliday'] = dfinput['pickup_datetime'].isin(holidays).astype(int)
    return dfinput

In [25]:
# TEST FUNCTIONS
# import file
year = 2019
dfsample_test = pd.read_csv('../data/'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

dfsample_test = clean_data(dfsample_test, year, dfManZon_array)
dfsample_test = add_date_features(dfsample_test, year)
dfsample_test.head()

,pickup_datetime,PULocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2019-04-29 05:00,90,1,2019,4,29,5,18,0,0,0
1,2019-04-29 06:00,68,1,2019,4,29,6,18,0,0,0
2,2019-04-29 06:00,107,1,2019,4,29,6,18,0,0,0
3,2019-04-29 06:00,113,1,2019,4,29,6,18,0,0,0
4,2019-04-29 06:00,132,1,2019,4,29,6,18,0,0,0


In [56]:
# FUNCTION TO RUN FOR EACH YEAR DATASET

def process_data_sample(yearIN, manhattan_zones_array):
    
    # 1. Get iterator with 3000 chunksize
    dfsi = pd.read_csv('../data/'+str(yearIN)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
    
    # 2. Declare empty DataFrame to store results of each chunk
    all_chunks=pd.DataFrame()

    # 3. Iterate through chunks and append results
    for i, dfs in enumerate(dfsi):
        print(i, len(dfs))

        # clean dataset
        result = clean_data(dfs, yearIN, manhattan_zones_array)

        all_chunks=all_chunks.append(result)

    # The chunks may be just part of a single day so I need to sort and groupby again
    result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                        groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    # add date features to dataset
    add_date_features(result_all, yearIN)
    
    # save csv
    result_all.to_csv('../data/Data_Taxis_'+str(yearIN)+'_Cleaned_TEST.csv', index = False, header=True)
    return result_all

In [60]:
dfsample_test = process_data_sample(2019,dfManZon_array)
dfsample_test

0 3000
1 3000
2 3000
3 999


,pickup_datetime,PULocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2019-04-29 05:00,90,1,2019,4,29,5,18,0,0,0
1,2019-04-29 06:00,68,1,2019,4,29,6,18,0,0,0
2,2019-04-29 06:00,107,1,2019,4,29,6,18,0,0,0
3,2019-04-29 06:00,113,1,2019,4,29,6,18,0,0,0
4,2019-04-29 06:00,132,1,2019,4,29,6,18,0,0,0
5,2019-04-29 06:00,138,1,2019,4,29,6,18,0,0,0
6,2019-04-29 06:00,236,1,2019,4,29,6,18,0,0,0
7,2019-04-29 06:00,237,1,2019,4,29,6,18,0,0,0
8,2019-04-29 07:00,68,1,2019,4,29,7,18,0,0,0
9,2019-04-29 07:00,113,1,2019,4,29,7,18,0,0,0


I check that the 'dfsample_test' dataframe and 'dfsample' are the same:
- Shape.
- Head and Tail.
- Taking a random row, they should be the same.

In [61]:
print(dfsample.shape)
print(dfsample_test.shape)

(141, 11)
(141, 11)


In [62]:
print(dfsample.tail())
print(dfsample_test.tail())

      pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  \
136  2019-04-29 13:00           229            1  2019      4   29    13   
137  2019-04-29 13:00           230            1  2019      4   29    13   
138  2019-04-29 13:00           231            1  2019      4   29    13   
139  2019-04-29 13:00           234            1  2019      4   29    13   
140  2019-04-29 13:00           236            2  2019      4   29    13   

     week  dayofweek  isweekend  IsHoliday  
136    18          0          0          0  
137    18          0          0          0  
138    18          0          0          0  
139    18          0          0          0  
140    18          0          0          0  
      pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  \
136  2019-04-29 13:00           229            1  2019      4   29    13   
137  2019-04-29 13:00           230            1  2019      4   29    13   
138  2019-04-29 13:00           231          

In [64]:
print(dfsample.head())
print(dfsample_test.head())

    pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  week  \
0  2019-04-29 05:00            90            1  2019      4   29     5    18   
1  2019-04-29 06:00            68            1  2019      4   29     6    18   
2  2019-04-29 06:00           107            1  2019      4   29     6    18   
3  2019-04-29 06:00           113            1  2019      4   29     6    18   
4  2019-04-29 06:00           132            1  2019      4   29     6    18   

   dayofweek  isweekend  IsHoliday  
0          0          0          0  
1          0          0          0  
2          0          0          0  
3          0          0          0  
4          0          0          0  
    pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  week  \
0  2019-04-29 05:00            90            1  2019      4   29     5    18   
1  2019-04-29 06:00            68            1  2019      4   29     6    18   
2  2019-04-29 06:00           107            1  2019     

In [65]:
# CHECK A RANDOM ROW
random_row=random.randint(1,len(dfsample.index))
print(dfsample.loc[random_row])
print(dfsample_test.loc[random_row])

pickup_datetime    2019-04-29 11:00
PULocationID                    100
NoOfPickups                     105
year                           2019
month                             4
day                              29
hour                             11
week                             18
dayofweek                         0
isweekend                         0
IsHoliday                         0
Name: 51, dtype: object
pickup_datetime    2019-04-29 11:00
PULocationID                    100
NoOfPickups                     105
year                           2019
month                             4
day                              29
hour                             11
week                             18
dayofweek                         0
isweekend                         0
IsHoliday                         0
Name: 51, dtype: object


### 3. Run program with Big Data

In [48]:
# FUNCTION TO RUN FOR EACH YEAR DATASET

def process_data(yearIN, manhattan_zones_array):
    
    # 1. Get iterator with 3000 chunksize
    dfsi = pd.read_csv('../data/'+str(yearIN)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
    
    # 2. Declare empty DataFrame to store results of each chunk
    all_chunks=pd.DataFrame()

    # 3. Iterate through chunks and append results
    for i, dfs in enumerate(dfsi):
        print(i, len(dfs))

        # clean dataset
        result = clean_data(dfs, yearIN, manhattan_zones_array)

        all_chunks=all_chunks.append(result)

    # The chunks may be just part of a single day so I need to sort and groupby again
    result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                        groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    # add date features to dataset
    add_date_features(result_all, yearIN)
    
    # save csv
    result_all.to_csv('../data/Data_Taxis_'+str(yearIN)+'_Cleaned_TEST.csv', index = False, header=True)

In [47]:
process_data(2017,dfManZon_array)

0 3000
1 3000
2 3000
3 999


In [ ]:
process_data(2018,dfManZon_array)

In [ ]:
process_data(2019,dfManZon_array)

In [24]:
# PERSONAL NOTES:
# interpolate missing values every hour

### 4. Sanity Check
Let´s check that the csv file created is correct

In [67]:
year=2019

csvcheck = pd.read_csv('../data/Data_Taxis_'+str(year)+'_Cleaned.csv', sep=',',
                       parse_dates=['pickup_datetime'])

print('Year should be ' + str(year)+':', csvcheck.year.unique())
print('There should be 12 months:', csvcheck.month.nunique())
print('There should be 31 days:', csvcheck.day.nunique())
print('There should be 24 hours:', csvcheck.hour.nunique())
print('There should be 52 weeks:', csvcheck.week.nunique())

csvcheck['hourlyperiods'] = 1
h = csvcheck.groupby(['month','day','hour'])['hourlyperiods'].sum()
print('There should be 8759 hourly periods in a year: ', h.shape)
print(csvcheck.describe())

Year should be 2019: [2019]
There should be 12 months: 12
There should be 31 days: 31
There should be 24 hours: 24
There should be 52 weeks: 52
There should be 8759 hourly periods in a year:  (8759,)
       PULocationID   NoOfPickups       year         month           day  \
count  1.147178e+06  1.147178e+06  1147178.0  1.147178e+06  1.147178e+06   
mean   1.395355e+02  7.266744e+01     2019.0  6.454254e+00  1.565817e+01   
std    7.677165e+01  1.349620e+02        0.0  3.467905e+00  8.799955e+00   
min    1.000000e+00  1.000000e+00     2019.0  1.000000e+00  1.000000e+00   
25%    7.400000e+01  1.000000e+00     2019.0  3.000000e+00  8.000000e+00   
50%    1.420000e+02  7.000000e+00     2019.0  6.000000e+00  1.600000e+01   
75%    2.110000e+02  7.900000e+01     2019.0  9.000000e+00  2.300000e+01   
max    2.650000e+02  2.250000e+03     2019.0  1.200000e+01  3.100000e+01   

               hour          week     dayofweek     isweekend  IsHoliday  \
count  1.147178e+06  1.147178e+06  1.14